In [ ]:
import pandas as pd
import re
from enum import Enum

In [ ]:
df = pd.read_csv("./input.txt", header=None, sep=" ")
df = df.rename(columns={0: 'hand', 1: 'bid'})

In [ ]:
from collections import Counter

HandType = Enum('HandType', ['FiveOfAKind', 'FourOfAKind', 'FullHouse', 'ThreeOfaKind', 'TwoPair', 'Pair', 'HighCard'])

#J value toggled to 0 for task 2
cardValue = {
    "A": 13,
    "K": 12,
    "Q": 11,
    "J": 0,
    "T": 9,
    "9": 8,
    "8": 7,
    "7": 6,
    "6": 5,
    "5": 4,
    "4": 3,
    "3": 2,
    "2": 1,
}

handTypeValue = {
    HandType.FiveOfAKind: 7,
    HandType.FourOfAKind: 6,
    HandType.FullHouse: 5,
    HandType.ThreeOfaKind: 4,
    HandType.TwoPair: 3,
    HandType.Pair: 2,
    HandType.HighCard: 1,
}

def getHandType(hand):
    letterCounts = Counter(hand)
    #___TASK 2___
    #search for highest key would not return a result if there are 5 of them and it would not change the card type
    if("J" in letterCounts and letterCounts["J"] != 5):
        jCount = letterCounts.pop("J")
        #find letter in card that appears the most and add j count to that
        highestKey = ("",0)
        for key in letterCounts:
            if letterCounts[key]>highestKey[1]:
                highestKey = (key, letterCounts[key])
        letterCounts[highestKey[0]]+= jCount
    #___TASK 2 END___
    valueCounts = list(letterCounts.values())
    if 5 in valueCounts: return HandType.FiveOfAKind
    if 4 in valueCounts: return HandType.FourOfAKind
    if 3 in valueCounts and 2 in valueCounts: return HandType.FullHouse
    if 3 in valueCounts: return HandType.ThreeOfaKind
    if Counter(valueCounts)[2] == 2: return HandType.TwoPair
    if 2 in valueCounts: return HandType.Pair
    return HandType.HighCard

#a little bit hacky
df["type"] = df["hand"].apply(lambda x: getHandType(x))
df["typeValue"] = df["type"].apply(lambda x: handTypeValue[x])
df["firstLetterValue"] = df["hand"].apply(lambda x: cardValue[x[0]])
df["secondLetterValue"] = df["hand"].apply(lambda x: cardValue[x[1]])
df["thirdLetterValue"] = df["hand"].apply(lambda x: cardValue[x[2]])
df["fourthLetterValue"] = df["hand"].apply(lambda x: cardValue[x[3]])
df["fifthLetterValue"] = df["hand"].apply(lambda x: cardValue[x[4]])
ascending = True
df = df.sort_values(
    ["typeValue", "firstLetterValue", "secondLetterValue", "thirdLetterValue", "fourthLetterValue", "fifthLetterValue"],
    ascending=[True]*6
)
df = df.reset_index(drop=True)
df = df.reset_index()
df["value"] = df[["index", "bid"]].apply(lambda x: (x["index"]+1) * x["bid"], axis=1)
df["value"].sum()